# Cálculo de autovalores y autovectores
Los métodos implementados son los siguientes:
- Método de la potencia: `potencia`.
- Método de la potencia inversa: `potencia_inversa`.
- Método de la potencia desplazada: `potencia_desplazada`.
- Método de la potencia inversa desplazada: `potencia_inversa_desplazada`.

In [1]:
from numpy import *
from numpy.linalg import eig, norm, solve

## Problemas propuestos

### Problema 1
Hallar información sobre los autovectores de la matriz
$$A = \begin{pmatrix}
    -33 & 36 & -27 & 18 & -9 \\
    -33 & 51 & -45 & 30 & -15 \\
    -19 & 38 & -45 & 34 & -17 \\
    -9 & 18 & -27 & 27 & 15 \\
    -3 & 6 & -9 & 12 & -9
\end{pmatrix}.$$

In [2]:
A = array([
    [-33, 36, -27, 18, -9],
    [-33, 51, -45, 30, -15],
    [-19, 38, -45, 34, -17],
    [-9, 18, -27, 27, -15],
    [-3, 6, -9, 12, -9]
])

autovalores, autovectores = eig(A)
print("Autovalores\n", autovalores)
print("Autovectores\n", autovectores)

Autovalores
 [ 12.   6. -15.  -9.  -3.]
Autovectores
 [[ 4.08248290e-01 -7.67654634e-15 -8.94427191e-01  9.49240207e-15
   1.53647880e-15]
 [ 8.16496581e-01 -1.39299896e-14 -4.47213595e-01  4.08248290e-01
   2.89219539e-15]
 [ 4.08248290e-01 -4.08248290e-01 -1.08161513e-15  8.16496581e-01
   4.71990220e-15]
 [ 2.42114535e-16 -8.16496581e-01 -8.76481225e-16  4.08248290e-01
   4.47213595e-01]
 [ 9.77469767e-17 -4.08248290e-01 -3.72970734e-17 -1.32364172e-15
   8.94427191e-01]]


## Problema 2
Hallar todos los autovalores y autovectores de la matriz simétrica
$$B = \begin{pmatrix}
    0 & 1 & 6 & 0 & 0 & 0 \\
    1 & 0 & 2 & 7 & 0 & 0 \\
    6 & 2 & 0 & 3 & 8 & 0 \\
    0 & 7 & 3 & 0 & 4 & 9 \\
    0 & 0 & 8 & 4 & 0 & 5 \\
    0 & 0 & 0 & 9 & 5 & 0
\end{pmatrix}.$$

In [3]:
B = array([
    [0, 1, 6, 0, 0, 0],
    [1, 0, 2, 7, 0, 0],
    [6, 2, 0, 3, 8, 0],
    [0, 7, 3, 0, 4, 9],
    [0, 0, 8, 4, 0, 5],
    [0, 0, 0, 9, 5, 0]
])

autovalores, autovectores = eig(B)
print("Autovalores\n", autovalores)
print("Autovectores\n", autovectores)

Autovalores
 [ 16.60600885 -12.1283007  -10.0647204   -2.46535845   5.94293604
   2.10943466]
Autovectores
 [[ 0.16934678  0.22443065 -0.27393137  0.67488298 -0.55695938  0.28327259]
 [ 0.2919696  -0.27590355 -0.38964313 -0.32988958  0.14870065  0.74556703]
 [ 0.42003409 -0.40767648  0.52444763 -0.22232315 -0.57644577 -0.02467033]
 [ 0.54843355  0.56245193  0.44952646  0.0832942   0.37050991  0.19125615]
 [ 0.46622115  0.30778629 -0.52551546 -0.38641188 -0.18661939 -0.47707232]
 [ 0.43761314 -0.54426411 -0.14090415  0.47961044  0.40409189 -0.31480296]]


## Método de la potencia
Sea $A \in \mathcal{M}_n(\mathbb{C})$ y sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = Ax_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A$.

In [4]:
def potencia(A, x0, tolerancia = 1e-7, iteraciones = 100):
    x = x0
    
    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = A@x
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia al primer problema para hallar el autovector asociado al autovalor de mayor módulo.

In [5]:
x0 = array([1, 0, 0, 0, 0])
x = potencia(A, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[ 8.94427191e-01  4.47213595e-01 -6.78740694e-10 -3.15446760e-16
 -9.45290244e-17]
Asociado al autovalor -15.000000032782548


## Método de la potencia inversa
Sea $A \in \mathcal{M}_n(\mathbb{C})$ invertible y sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = A^{-1}x_k \Leftrightarrow Ay_{k+1} = x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A^{-1}$ o, equivalentemente, el autovalor de menor módulo de $A$.

In [6]:
def potencia_inversa(A, x0, tolerancia = 1e-7, iteraciones = 100):
    x = x0
    
    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = solve(A, x)
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa al primer problema para hallar el autovector asociado al autovalor de menor módulo.

In [7]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_inversa(A, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[-4.72615066e-16  5.26733081e-13 -1.33269496e-08  4.47213580e-01
  8.94427199e-01]
Asociado al autovalor -3.00000021457814


## Método de la potencia desplazada
Sea $A \in \mathcal{M}_n(\mathbb{C})$, sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera y sea $\mu \in \mathbb{C}$.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = (A - \mu I)x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A - \mu I$ o, equivalentemente, el autuvalor más lejano de $\mu$.

In [8]:
def potencia_desplazada(A, mu, x0, tolerancia = 1e-7, iteraciones = 100):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None
    x = x0

    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = (A - mu*eye(n))@x
        x = y/norm(y)

        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa desplazada al primer problema para hallar el autovector asociado al autovalor más lejano a -5.

In [9]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_desplazada(A, -5, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[-4.08248289e-01 -8.16496579e-01 -4.08248295e-01 -1.11860032e-08
 -5.59300154e-09]
Asociado al autovalor 11.999999999921512


## Método de la potencia inversa desplazada
Sea $A \in \mathcal{M}_n(\mathbb{C})$, sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera y sea $\mu \notin sp(A)$.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = (A - \mu I)^{-1}x_k \Leftrightarrow (A - \mu I)y_{k+1} = x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $(A - \mu I)^{-1}$ o, equivalentemente, el autovalor más cercano a $\mu$.

In [10]:
def potencia_inversa_desplazada(A, mu, x0, tolerancia = 1e-7, iteraciones = 100):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None
    x = x0

    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = solve(A - mu*eye(n), x)
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa desplazada al primer problema para hallar el autovector asociado al autovalor más cercano a 7.

In [11]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_inversa_desplazada(A, 7, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[-3.34436871e-09 -6.68873563e-09 -4.08248293e-01 -8.16496580e-01
 -4.08248290e-01]
Asociado al autovalor 6.000000008195061


## Método de Jacobi
Sea $A \in \mathcal{M}_n(\mathbb{R})$ simétrica y sea $A_1 = A$.
Para $k = 1, 2, \dots$, elegimos el par $(p, q)$ de forma que
$$|a^k_{pq}| = \max_{i \neq j} |a^k_{ij}|$$
y construimos
$$\Omega_k = \begin{pmatrix}
    1 \\
    & \ddots \\
    & & \cos(\theta_k) & \dots & \sin(\theta_k) \\
    & & \vdots & \ddots & \vdots \\
    & & -\sin(\theta_k) & \dots & \cos(\theta_k) \\
    & & & & & \ddots \\
    & & & & & & 1
\end{pmatrix},$$
donde
\begin{align*}
    \omega^k_{pp} & = \omega^k_{pp} = \cos(\theta_k), \\
    \omega^k_{pq} & = \sin(\theta_k), \\
    \omega^k_{qp} & = -\sin(\theta_k), \\
    \omega^k_{ii} & = 1, \quad \text{si } i \neq p, \\
    \omega^k_{ij} & = 0, \quad \text{en el resto},
\end{align*}
con $\theta_k \in (-\frac{\pi}{4}, \frac{\pi}{4}] \setminus \{0\}$ tal que
$$cotg(2\theta_k) = \frac{a^k_{qq} - a^k_{pp}}{2a^k_{pq}}.$$
El método de Jacobi toma entonces
$$A_{k+1} = \Omega_k^tA\Omega_k.$$

La sucesión $\{A_k\}$ converge a una matriz diagonal cuyos elementos diagonales son los autovalores de $A$.
Si además todos los autovalores son distintos, entonces la sucesión $\{O_k\}$, donde $O_k = \Omega_1\dots\Omega_k$, converge a una matriz ortogonal $O$ cuyas columnas forman una base ortonormal de autovectores de $A$.

In [12]:
def jacobi(A, tolerancia = 1e-7, iteraciones = 100):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None

    Ak = array(A, dtype = float)
    mascara = ones((n, n))
    fill_diagonal(mascara, 0)
    Ok = eye(n)

    iteracion = 0
    maximo = abs(Ak*mascara).max()

    while iteracion < iteraciones and maximo > tolerancia:
        posicion_maxima = argmax(abs(Ak*mascara))
        p = int(posicion_maxima/n)
        q = posicion_maxima % n

        if abs(Ak[p, p] - Ak[q, q]) < 1e-15:
            theta = pi/4
        else:
            theta = arctan(2*Ak[p, q]/(Ak[q, q] - Ak[p, p]))/2

        Omega = eye(n)
        Omega[p, p] = cos(theta)
        Omega[p, q] = sin(theta)
        Omega[q, p] = -Omega[p, q]
        Omega[q, q] = Omega[p, p]

        Bk = transpose(Omega)@Ak@Omega
        Ak = copy(Bk)
        Ok = Ok@Omega

        maximo = abs(Ak*mascara).max()
        iteracion += 1

    return diag(Ak), Ok

Aplicamos el método de Jacobi al segundo problema para hallar todos los autovalores de la matriz.

In [13]:
autovalores, autovectores = jacobi(B)
print("Autovalores\n", autovalores)
print("Autovectores\n", autovectores)

Autovalores
 [ -2.46535845   2.10943466 -10.0647204  -12.1283007    5.94293604
  16.60600885]
Autovectores
 [[ 0.67488298  0.28327261 -0.27393137  0.22443065  0.55695938  0.16934678]
 [-0.32988959  0.74556703 -0.38964313 -0.27590355 -0.14870065  0.2919696 ]
 [-0.22232315 -0.02467034  0.52444763 -0.40767648  0.57644577  0.42003408]
 [ 0.08329419  0.19125615  0.44952646  0.56245193 -0.37050991  0.54843355]
 [-0.38641187 -0.47707232 -0.52551546  0.30778629  0.18661939  0.46622115]
 [ 0.47961045 -0.31480295 -0.14090415 -0.54426411 -0.40409189  0.43761314]]
